
# NaviGator Toolkit API test

This notebook is designed to test connection to the NaviGator Toolkit API ([see here for more information](https://it.ufl.edu/ai/navigator-toolkit/)). You will need a NaviGator API key. The key should be stored in a `.json` file with the following format:

    {
      "OPENAI_API_KEY" : "Put your key here in the quotes",
      "base_url" : "https://api.ai.it.ufl.edu/"
    }

We suggest putting that in your home directory, to minimize the chances of accidentally adding and committing the file to a git repo. Remember that anyone with your API key can use NaviGator as you! 

In [1]:
import openai
import os
import json
import subprocess

## Load `.json` file with your key and api endpoint URL

In [2]:
# Set the path to your jsnkey file
key_file = '/home/magitz/navigator_api_keys.json'


# Load the JSON file
with open(key_file, 'r') as file:
    data = json.load(file)

# Extract the values
OPENAI_API_KEY = data.get('OPENAI_API_KEY')
base_url = data.get('base_url')

# Set the environment variable
os.environ['TOOLKIT_API_KEY'] = OPENAI_API_KEY


## Test connectivity and get model list

Reply should list the models that are available with your API key. An example, truncated, output is:

    SyncPage[Model](data=[Model(id='llama-3.1-70b-instruct', created=1677610602, object='model', owned_by='openai'), Model(id='sfr-embedding-mistral', created=1677610602, object='model', owned_by='openai'),...)], object='list')

In [3]:
# Check list of available models
client = openai.OpenAI(
    api_key=os.environ.get("TOOLKIT_API_KEY"),
    base_url=base_url
)

response = client.models.list()
 
print(response)

SyncPage[Model](data=[Model(id='llama-3.3-70b-instruct', created=1677610602, object='model', owned_by='openai'), Model(id='nim-llama-3.1-8b-instruct', created=1677610602, object='model', owned_by='openai'), Model(id='llama-3.1-8b-instruct', created=1677610602, object='model', owned_by='openai'), Model(id='whisper-large-v3', created=1677610602, object='model', owned_by='openai'), Model(id='sfr-embedding-mistral', created=1677610602, object='model', owned_by='openai'), Model(id='flux.1-dev', created=1677610602, object='model', owned_by='openai'), Model(id='mixtral-8x7b-instruct', created=1677610602, object='model', owned_by='openai'), Model(id='llama-3.1-70b-instruct', created=1677610602, object='model', owned_by='openai'), Model(id='nomic-embed-text-v1.5', created=1677610602, object='model', owned_by='openai'), Model(id='gte-large-en-v1.5', created=1677610602, object='model', owned_by='openai'), Model(id='codestral-22b', created=1677610602, object='model', owned_by='openai'), Model(id='

## Get information about the key

Use the API to get key information. Note that this code intentionally skipt printing the `key` part of the output for securit.

In [4]:
# Set up and run the curl command
key_info_cmd=f"curl -X 'GET' '{base_url}key/info?key={os.environ.get('TOOLKIT_API_KEY')}' -H 'accept: application/json' -H 'Authorization: Bearer {os.environ.get('TOOLKIT_API_KEY')}'"
result=subprocess.run(key_info_cmd, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Parse the reply, printing the `info` part of the reply nicely 
if result.returncode != 0:
    print("Error:", result.stderr.decode('utf-8'))
else:
    response = result.stdout.decode('utf-8')
    response_dict = json.loads(response)
    
    # Access the "info" section
    info_section = response_dict.get('info', {})
    
    # Print or use the "info" section
    print(json.dumps(info_section, indent=4))  # Pretty-print the "info" section

{
    "key_name": "sk-...LbYQ",
    "key_alias": "Matt_2024",
    "soft_budget_cooldown": false,
    "spend": 0.003963,
    "expires": "2025-12-11T21:48:41.470000+00:00",
    "models": [
        "all-team-models"
    ],
    "aliases": {},
    "config": {},
    "user_id": "magitz@ufl.edu",
    "team_id": "a2cc7649-8b8f-4441-88f1-7360c056adc7",
    "permissions": {},
    "max_parallel_requests": null,
    "metadata": {
        "tags": null,
        "guardrails": []
    },
    "blocked": null,
    "tpm_limit": null,
    "rpm_limit": null,
    "max_budget": 5.0,
    "budget_duration": "30d",
    "budget_reset_at": "2025-04-15T18:39:53.589000+00:00",
    "allowed_cache_controls": [],
    "model_spend": {},
    "model_max_budget": {},
    "budget_id": null,
    "created_at": "2024-12-11T21:48:41.472000+00:00",
    "updated_at": "2025-03-17T15:43:05.641000+00:00",
    "litellm_budget_table": null
}


In [19]:
print(f"You have spent: ${info_section['spend']} of the key's ${info_section['max_budget']:.2f} monthly limit, leaving ${info_section['max_budget'] -info_section['spend']} to spend this month.")

You have spent: $0.003963 of the key's $5.00 monthly limit, leaving $4.996037 to spend this month.
